## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210401.ods'

In [5]:
try:
    datos_row = pd.read_excel(enlace,sheet_name='Comunicación')
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210401.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [6]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1098720,168000,381600,1648320,1483868,0.900231,514632,2021-03-31
1,Aragón,209745,35300,59900,304945,258902,0.849012,93627,2021-03-31
2,Asturias,203315,32700,46000,282015,234491,0.831484,89604,2021-03-31
3,Baleares,112680,18000,52700,183380,160884,0.877326,45509,2021-03-30
4,Canarias,232800,35800,98100,366700,315167,0.859468,108593,2021-03-31
5,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,2021-03-30
6,Castilla y Leon,463035,78300,108100,649435,554254,0.853440,207364,2021-03-31
7,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,2021-03-30
8,Cataluña,1027880,168400,350800,1547080,1353057,0.874588,431509,2021-03-31
9,C. Valenciana,586360,96800,228100,911260,821536,0.901539,257143,2021-03-31


..hacemos una copia para hacerle las transformaciones necesarias:

In [7]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [8]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [9]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [10]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [11]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [12]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,1098720,168000,381600,1648320,1483868,0.900231,514632,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,209745,35300,59900,304945,258902,0.849012,93627,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,203315,32700,46000,282015,234491,0.831484,89604,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,112680,18000,52700,183380,160884,0.877326,45509,30/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,232800,35800,98100,366700,315167,0.859468,108593,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,30/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,463035,78300,108100,649435,554254,0.853440,207364,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,30/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,1027880,168400,350800,1547080,1353057,0.874588,431509,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,586360,96800,228100,911260,821536,0.901539,257143,31/3/21,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [13]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [14]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [15]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [16]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,1/4/21,Andalucía,1098720,168000,381600,1648320,1483868,0.900231,514632,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,1/4/21,Aragón,209745,35300,59900,304945,258902,0.849012,93627,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,1/4/21,Asturias,203315,32700,46000,282015,234491,0.831484,89604,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,1/4/21,Baleares,112680,18000,52700,183380,160884,0.877326,45509,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,1/4/21,Canarias,232800,35800,98100,366700,315167,0.859468,108593,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,1/4/21,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,1/4/21,Castilla y Leon,463035,78300,108100,649435,554254,0.853440,207364,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,1/4/21,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,1/4/21,Cataluña,1027880,168400,350800,1547080,1353057,0.874588,431509,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,1/4/21,C. Valenciana,586360,96800,228100,911260,821536,0.901539,257143,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [17]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [18]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [19]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,31/3/21,País Vasco,314550.0,52900.0,100200.0,467650,370971,0.793266,105534.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1222,31/3/21,Ceuta,7830.0,1300.0,3900.0,13030,11226,0.861550,3601.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1223,31/3/21,Melilla,7830.0,1200.0,3900.0,12930,10413,0.805336,2423.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1224,31/3/21,Fuerzas Armadas,19500.0,0.0,36100.0,55600,37606,0.676367,4991.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [20]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [21]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [22]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [23]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,1/4/21,País Vasco,314550.0,52900.0,100200.0,467650,370971,0.793266,105534.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1243,1/4/21,Ceuta,7830.0,1300.0,3900.0,13030,11226,0.861550,3601.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1244,1/4/21,Melilla,7830.0,1200.0,3900.0,12930,10413,0.805336,2423.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1245,1/4/21,Fuerzas Armadas,19500.0,0.0,36100.0,55600,40099,0.721205,4991.0,NaN,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [24]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [25]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Institucionalizados

In [26]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_institucionalizados_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Institucionalizados'

In [27]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [28]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210401.ods


In [29]:
datos_row

,Unnamed: 0,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA
0,Andalucía,41651,40770,38848,0.978848,0.932703
1,Aragón,17829,17829,17025,1.000000,0.954905
2,Asturias,11948,11577,11115,0.968949,0.930281
3,Baleares,5496,5117,4869,0.931041,0.885917
4,Canarias,10531,10531,9441,1.000000,0.896496
5,Cantabria,6919,6821,6429,0.985836,0.929181
6,Castilla y Leon,45800,45717,43930,0.998188,0.959170
7,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378
8,Cataluña,73343,73068,62908,0.996250,0.857723
9,C. Valenciana,29500,29240,26788,0.991186,0.908068


In [30]:
datos_hoy = datos_row.copy()

In [31]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [32]:
datos_hoy.columns = ['ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA', 'date_pub',
       'source_name', 'source']

In [33]:
datos_hoy

,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,date_pub,source_name,source
0,Andalucía,41651,40770,38848,0.978848,0.932703,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,17829,17829,17025,1.000000,0.954905,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,11948,11577,11115,0.968949,0.930281,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,5496,5117,4869,0.931041,0.885917,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,10531,10531,9441,1.000000,0.896496,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,6919,6821,6429,0.985836,0.929181,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,45800,45717,43930,0.998188,0.959170,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,73343,73068,62908,0.996250,0.857723,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,29500,29240,26788,0.991186,0.908068,1/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [34]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA',
       'source_name', 'source']]

In [35]:
datos_hoy

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,1/4/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,1/4/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,1/4/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,1/4/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,1/4/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,1/4/21,Cantabria,6919,6821,6429,0.985836,0.929181,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,1/4/21,Castilla y Leon,45800,45717,43930,0.998188,0.959170,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,1/4/21,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,1/4/21,Cataluña,73343,73068,62908,0.996250,0.857723,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,1/4/21,C. Valenciana,29500,29240,26788,0.991186,0.908068,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [36]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_20210401'

In [37]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

..y ahora lo merge a los acumulados::

In [38]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [39]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,6919,6821,6429,0.985836,0.929181,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,45800,45717,43930,0.998188,0.959170,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,73343,73068,62908,0.996250,0.857723,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,29500,29240,26788,0.991186,0.908068,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [40]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [41]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,6919,6821,6429,0.985836,0.929181,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,45800,45717,43930,0.998188,0.959170,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,73343,73068,62908,0.996250,0.857723,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,29500,29240,26788,0.991186,0.908068,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [42]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias ', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon ', 'Castilla - La Mancha',
       'Cataluña', 'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja',
       'Madrid', 'Murcia ', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Totales'], dtype=object)

y volvemos a subirlo:

In [43]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Etarios 1 dosis

In [55]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_1dosis_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=3,day=31)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_al_menos_1_dosis'

In [56]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [57]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210401.ods


In [58]:
datos_hoy = datos_row.copy()

In [59]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [60]:
datos_hoy.columns = ['ccaa', 'Personas con al menos 1 dosis ≥80 años',
       'Población INE≥80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [61]:
datos_hoy = datos_hoy [['date_pub','ccaa', 'Personas con al menos 1 dosis ≥80 años',
       'Población INE≥80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 
       'source_name', 'source']]

In [62]:
datos_hoy

,date_pub,ccaa,Personas con al menos 1 dosis ≥80 años,Población INE≥80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,1/4/21,Andalucía,391814,413516,0.947518,27887,650381,0.042878,70060,916147,...,625846,0.040730,152,181921,0.000836,969236,7062213,0.137243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,1/4/21,Aragón,73468,98807,0.743551,3683,118955,0.030961,12136,156367,...,89180,0.061920,28,25106,0.001115,165275,1132764,0.145904,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,1/4/21,Asturias,71508,87826,0.814201,4360,109249,0.039909,16775,149652,...,54179,0.053176,30,15563,0.001928,144887,901209,0.160770,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,1/4/21,Baleares,33141,49542,0.668948,2346,80516,0.029137,15295,117231,...,85839,0.040972,31,23300,0.001330,115375,986279,0.116980,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,1/4/21,Canarias,56952,90236,0.631145,16979,154531,0.109874,36450,235092,...,163899,0.034180,28,44352,0.000631,206574,1871033,0.110406,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,1/4/21,Cantabria,30858,41042,0.751864,1827,53066,0.034429,4962,77345,...,34935,0.040332,5,10791,0.000463,70024,501384,0.139661,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,1/4/21,Castilla y Leon,164648,223657,0.736163,14488,245410,0.059036,27302,318575,...,144422,0.049161,89,40684,0.002188,346890,2092873,0.165748,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,1/4/21,Castilla - La Mancha,99503,135194,0.736002,6609,159101,0.041540,17253,218444,...,149518,0.037012,91,42364,0.002148,233231,1719964,0.135602,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,1/4/21,Cataluña,285022,447196,0.637354,38673,635292,0.060874,159902,837584,...,556316,0.060394,279,159864,0.001745,921548,6531658,0.141089,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,1/4/21,C. Valenciana,232661,276531,0.841356,12527,437862,0.028609,36414,580728,...,352834,0.044049,106,103049,0.001029,564393,4269305,0.132198,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [63]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_20210401'

In [64]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora merge con los acumulados:

In [65]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [67]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis ≥80 años,Población INE≥80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,30858,41042,0.7518639442522295,1827,53066,0.03442882448271963,4962,77345,...,34935,0.04033204522684986,5,10791,0.0004633490872022982,70024,501384,0.13966141719719816,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,158345,223657,0.7079814179748454,12861,245410,0.052406177417383154,26304,318575,...,144422,0.04880835329797399,89,40684,0.002187592173827549,336070,2092873,0.16057830551591043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,99503,135194,0.7360015977040402,6609,159101,0.041539650913570626,17253,218444,...,149518,0.037012266081675786,91,42364,0.0021480502313284866,233231,1719964,0.1356022567914212,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,240692,447196,0.5382248499539352,36372,635292,0.0572524130635991,142711,837584,...,556316,0.059850876120765895,270,159864,0.0016889355952559675,853901,6531658,0.13073265624134026,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,212048,276531,0.7668145705183144,11809,437862,0.026969684512471965,33846,580728,...,352834,0.04036742490803041,106,103049,0.0010286368620753233,516703,4269305,0.12102742718077064,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [68]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [69]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis ≥80 años,Población INE≥80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,30858,41042,0.7518639442522295,1827,53066,0.03442882448271963,4962,77345,...,34935,0.04033204522684986,5,10791,0.0004633490872022982,70024,501384,0.13966141719719816,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,158345,223657,0.7079814179748454,12861,245410,0.052406177417383154,26304,318575,...,144422,0.04880835329797399,89,40684,0.002187592173827549,336070,2092873,0.16057830551591043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,99503,135194,0.7360015977040402,6609,159101,0.041539650913570626,17253,218444,...,149518,0.037012266081675786,91,42364,0.0021480502313284866,233231,1719964,0.1356022567914212,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,240692,447196,0.5382248499539352,36372,635292,0.0572524130635991,142711,837584,...,556316,0.059850876120765895,270,159864,0.0016889355952559675,853901,6531658,0.13073265624134026,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,212048,276531,0.7668145705183144,11809,437862,0.026969684512471965,33846,580728,...,352834,0.04036742490803041,106,103049,0.0010286368620753233,516703,4269305,0.12102742718077064,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [70]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Etarios con pauta completa

In [71]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_pauta_completa_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_pauta_completa'

In [72]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [73]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210401.ods


In [74]:
datos_hoy = datos_row.copy()

In [75]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [76]:
datos_hoy.columns = ['ccaa', 'Personas pauta completa ≥80 años',
       'Población INE≥80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [77]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Personas pauta completa ≥80 años',
       'Población INE≥80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE',
       'source_name', 'source']]

In [78]:
datos_hoy

,date_pub,ccaa,Personas pauta completa ≥80 años,Población INE≥80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,1/4/21,Andalucía,246229,413516,0.595452,13404,650381,0.020609,41934,916147,...,625846,0.018821,83,181921,0.000456,514632,7062213,0.072871,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,1/4/21,Aragón,44314,98807,0.448490,3135,118955,0.026355,9329,156367,...,89180,0.016035,25,25106,0.000996,93627,1132764,0.082654,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,1/4/21,Asturias,39438,87826,0.449047,3575,109249,0.032723,9848,149652,...,54179,0.042784,25,15563,0.001606,89604,901209,0.099426,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,1/4/21,Baleares,12403,49542,0.250353,2088,80516,0.025933,4856,117231,...,85839,0.014318,30,23300,0.001288,45509,986279,0.046142,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,1/4/21,Canarias,33459,90236,0.370794,12535,154531,0.081116,9116,235092,...,163899,0.014381,19,44352,0.000428,108593,1871033,0.058039,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,1/4/21,Cantabria,22177,41042,0.540349,1660,53066,0.031282,3704,77345,...,34935,0.017003,3,10791,0.000278,44056,501384,0.087869,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,1/4/21,Castilla y Leon,104815,223657,0.468642,9036,245410,0.036820,19853,318575,...,144422,0.017864,82,40684,0.002016,207364,2092873,0.099081,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,1/4/21,Castilla - La Mancha,46971,135194,0.347434,5152,159101,0.032382,12955,218444,...,149518,0.021609,74,42364,0.001747,122854,1719964,0.071428,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,1/4/21,Cataluña,152848,447196,0.341792,25780,635292,0.040580,46438,837584,...,556316,0.028169,190,159864,0.001189,431509,6531658,0.066064,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,1/4/21,C. Valenciana,119347,276531,0.431586,9679,437862,0.022105,22735,580728,...,352834,0.014409,85,103049,0.000825,257143,4269305,0.060231,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [79]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_20210401'

In [80]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora los merge con los acumulados

In [81]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [82]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [83]:
datos_acumulados

,date_pub,ccaa,Personas pauta completa ≥80 años,Población INE≥80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,240454,413516,0.5814865688389325,12788,650381,0.019662321008762557,41807,916147,...,625846,0.018677118652192393,83,181921,0.0004562419951517417,507430,7062213,0.0718514154132706,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,40245,98807,0.40730919874097987,3048,118955,0.025623134798873525,9273,156367,...,89180,0.015934065934065933,25,25106,0.0009957779016968055,89278,1132764,0.07881429847699963,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,37353,87826,0.4253068567394621,3490,109249,0.031945372497688765,9709,149652,...,54179,0.04263644585540523,24,15563,0.0015421191287026924,87093,901209,0.09664018002483331,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,12403,49542,0.2503532356384482,2088,80516,0.025932733866560684,4856,117231,...,85839,0.014317501368841669,30,23300,0.0012875536480686696,45509,986279,0.04614211597326923,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,31183,90236,0.34557161221685356,11216,154531,0.07258090609651138,9016,235092,...,163899,0.014331997144582944,19,44352,0.0004283910533910534,104712,1871033,0.055964806606831626,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,22177,41042,0.5403489108717898,1660,53066,0.03128180002261335,3704,77345,...,34935,0.017003005581794763,3,10791,0.00027800945232137893,44056,501384,0.08786877921912147,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,96294,223657,0.43054319784312584,8627,245410,0.03515341673118455,19766,318575,...,144422,0.01773967955020703,76,40684,0.0018680562383246485,198064,2092873,0.09463737168953873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,46971,135194,0.34743405772445524,5152,159101,0.03238194605942137,12955,218444,...,149518,0.021609438328495566,74,42364,0.0017467661221792087,122854,1719964,0.0714282391957041,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,145808,447196,0.3260494279913058,24886,635292,0.03917253798253401,45494,837584,...,556316,0.027939156882059838,188,159864,0.0011759995996597108,421043,6531658,0.06446188701245534,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,114569,276531,0.4143079799371499,9437,437862,0.02155245259922076,22540,580728,...,352834,0.014338187362895867,83,103049,0.0008054420712476589,251514,4269305,0.05891216486055693,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [84]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_'

In [85]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)